In [ ]:
import torch
from torch import nn

from torchvision.models.detection.backbone_utils import resnet_fpn_backbone

In [ ]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# Backbone

In [ ]:
backbone = resnet_fpn_backbone(
    'resnet18', # resnet18, resnet50
    pretrained=True,
    trainable_layers=5, # all layers
    # trainable_layers=3,
    returned_layers=[2,3,4]
)

In [ ]:
test_tensor = torch.randn(1,3,300,300)

In [ ]:
feature_dict = backbone(test_tensor)

In [ ]:
feature_dict.keys()

In [ ]:
features = list(feature_dict.values())

In [ ]:
for f in features:
    print(f.shape)

# Data

In [ ]:
from dataset.penn_fudan.penn_fudan_dataset import PennFudanDataset
from dataset.penn_fudan.draw_utils import visualize_dataset

In [ ]:
PENN_FUDAT_ROOT_PATH = 'dataset/penn_fudan/PennFudanPed'
dataset_train = PennFudanDataset(PENN_FUDAT_ROOT_PATH, train=True)

In [ ]:
img, mask, bboxs = dataset_train[0]
img.shape, mask.shape, bboxs.shape

In [ ]:
visualize_dataset(dataset_train)

# Model

In [ ]:
from centernet import create_model

In [ ]:
net = create_model()

In [ ]:
img, mask, bboxs = dataset_train[0]
img.shape, mask.shape, bboxs.shape

In [ ]:
out = net(img)